In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-19 01:24:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.76MB/s    in 0.2s    

2021-04-19 01:24:02 (5.76 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [9]:
high_votes_df = vine_df.filter(vine_df['total_votes'] >= 20)
high_votes_df

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [31]:
import pyspark.sql.functions as func
upper_half = high_votes_df.withColumn('helpful_vote_percent', func.round(((high_votes_df.helpful_votes/high_votes_df.total_votes)*100),0).cast('integer'))
upper_half.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|                  95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|                  95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|                 100|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|                  27|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|                  95|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|                  86|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|                  96|
|R2X8FZRUO

In [34]:

upper_half_df = upper_half.filter(upper_half['helpful_vote_percent'] >= 50)
upper_half_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|                  95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|                  95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|                 100|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|                  95|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|                  86|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|                  96|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|                  96|
|R2D8IMBVX

In [35]:
vine_y_df = upper_half.filter(upper_half['vine'] == 'Y')
vine_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|                  94|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|                  87|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|                  95|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|                  72|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|                  98|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|                  96|
|R1FNVUXPU63WOZ|          4|           43|         48|   Y|                N|                  90|
|R25XGG2G1

In [53]:
from pyspark.sql.functions import count

total_y_reviews = vine_y_df.select(count('review_id').alias("total_y_review_count"))
total_y_reviews.show()

+--------------------+
|total_y_review_count|
+--------------------+
|                  47|
+--------------------+



In [55]:
five_star_y_reviews = vine_y_df.filter(vine_y_df['star_rating'] == 5).select(count('review_id').alias("five_star_y_review_count"))
five_star_y_reviews.show()

+------------------------+
|five_star_y_review_count|
+------------------------+
|                      15|
+------------------------+



In [68]:
percent_y_reviews = round(((15/47)*100),0)
percent_y_reviews

32.0

In [36]:
vine_n_df = upper_half.filter(upper_half['vine'] == 'N')
vine_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|                  95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|                  95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|                 100|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|                  27|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|                  95|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|                  86|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|                  96|
|R2X8FZRUO

In [69]:
total_n_reviews = vine_n_df.select(count('review_id').alias("total_n_review_count"))
total_n_reviews.show()

+--------------------+
|total_n_review_count|
+--------------------+
|                9584|
+--------------------+



In [71]:
five_star_n_reviews = vine_n_df.filter(vine_n_df['star_rating'] == 5).select(count('review_id').alias("five_star_n_review_count"))
five_star_n_reviews.show()

+------------------------+
|five_star_n_review_count|
+------------------------+
|                    4516|
+------------------------+



In [72]:
percent_y_reviews = round(((4516/9584)*100),0)
percent_y_reviews

47.0